# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
<br>Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

__Цель исследования__
<br>Проанализировать базу данных для формирования ценностного предложения для нового продукта.

__Описание данных__

__Таблица `books`__<br>
Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

__Таблица `authors`__<br>
Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

__Таблица `publishers`__
Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

__Таблица `ratings`__
Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

__Таблица `reviews`__
Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.


__Задания__

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Открыть файлы с данными и изучить общую информацию

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

# убираем ограничения по столбцам и строкам
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.2f}'.format

# устанавливаем параметры
db_config = {
    'user': '', # имя пользователя
    'pwd': '', # пароль
    'host': '',
    'port': , # порт подключения
    'db': '' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [2]:
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

In [3]:
# создадим функцию, формирующую запрос, и выводящую для посмотра содержимое таблицы, общей информации, 
# количества пропусков, описательной статистики и количества явных дубликатов

def general_info(tab):
    query = '''SELECT * FROM {}'''.format(tab)
    df = get_sql_data(query)
    display(df.head())
    print('___________________________')
    print('')
    display(df.info())
    print('___________________________')
    print('')
    display(df.isna().sum())
    print('___________________________')
    print('')
    display(df.describe().T)
    print('___________________________')
    print('')
    print('Количество дубликатов:', df.duplicated().sum())
    print('___________________________')

In [4]:
# создаем список таблиц
tables = ['books','authors', 'publishers', 'ratings', 'reviews']

# с помощью цикла выведем информацию таблиц, используя функцию general_info()
for table in tables:
    print('')
    print('Таблица', table)
    general_info(table)


Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


___________________________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

___________________________



...

___________________________



review_id    0
book_id      0
username     0
text         0
dtype: int64

___________________________



,count,mean,std,min,25%,50%,75%,max
review_id,"2,793.00","1,397.00",806.41,1.00,699.00,"1,397.00","2,095.00","2,793.00"
book_id,"2,793.00",504.69,288.47,1.00,259.00,505.00,753.00,"1,000.00"


___________________________

Количество дубликатов: 0
___________________________


По итогу:

- явные дубликаты и пропуски отсутствуют,
- таблица `books` содержит 1000 строк,
- таблица `authors` содержит 636 строк,
- таблица `publishers` содержит 340 строк,
- таблица `ratings` содержит 6456 строк,
- таблица `reviews` содержит 2793 строки.

## Посчитаем, сколько книг вышло после 1 января 2000 года

In [5]:
query = '''
SELECT COUNT(*) 
FROM books 
WHERE publication_date > '2000-01-01'
'''
get_sql_data(query)

,count
0,819


После 1 января 2020 года вышло 819 книг

## Посчитаем количество обзоров и среднюю оценку для каждой книги

In [6]:
query = """
SELECT title,
       COUNT(DISTINCT review_id) AS cnt_review,
       ROUND(AVG(rating), 2) AS avg_rating
FROM books AS b 
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
GROUP BY b.book_id
ORDER BY avg_rating DESC,
         cnt_review DESC
"""
get_sql_data(query)

,title,cnt_review,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twenty-first Century,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


Самый высокий рейтинг 5 баллов у книги `A Dirty Job (Grim Reaper #1)` имеет 4 обзора.
<br>Самый низкий (1,5 балла) у книги с 2 обзорами - `Harvesting the Heart`.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [7]:
query = '''
SELECT p.publisher,
       COUNT(b.book_id)
FROM publishers AS p
LEFT JOIN books AS b ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY count DESC
LIMIT 3
'''
get_sql_data(query)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


Издательство, выпустившее наибольшее количество книг толще 50 страниц, только одно - `Penguin Books` (42 книги).

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [8]:
query = '''
SELECT a.author,
       AVG(rt.rating)
FROM authors AS a
JOIN books AS b ON b.author_id = a.author_id
JOIN ratings AS rt ON rt.book_id = b.book_id
WHERE b.book_id IN (SELECT b.book_id
                    FROM books AS b
                    JOIN ratings AS rt ON rt.book_id = b.book_id
                    GROUP BY b.book_id
                    HAVING COUNT(rt.rating_id) >= 50)
GROUP BY a.author
ORDER BY AVG DESC
LIMIT 3
'''
get_sql_data(query)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.29
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.25


Самая высокая средняя оценка книг 4,29 у автора `J.K. Rowling/Mary GrandPré`.

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [9]:
query = '''
WITH 
a AS (SELECT username,
             COUNT(review_id) AS cnt_review
      FROM reviews
      WHERE username IN (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating_id) > 48)
      GROUP BY username)
SELECT (SUM(cnt_review)/COUNT(cnt_review)) AS AVG
FROM a            
'''
get_sql_data(query)

,avg
0,24.00


В среднем на пользователя, которые поставили от 48 оценок и более, приходится 24 обзора.

__Вывод__
 
Из проведенного анализа базы данных мы выяснили, что:
- в сервисе хранится 1000 книг от 636 авторов и 340 издательств,
- 819 книг были выпущены с начала 2000 года и позже,
- средняя оценка книг от 1,5 до 5 баллов,
- больше всего книг выпустило издательство Penguin Books,
- J.K.Rowling/Mary GrandPré является автором с самой высокой оценкой книг,
- в среднем 24 обзора приходится на самых активных пользователей.